In [27]:
import pandas as pd
import datetime
import pandas as pd
from sklearn import preprocessing
import warnings
import datetime
import pickle
import sqlalchemy as sa
import pandas as pd
import data.constant as con
from tqdm import tqdm
import component.create_raceID as cr
import classes.db_operation_class as db

In [30]:
class Main():
    # コンストラクタ dbのインスタンス作成
    def __init__(self):
        #pickleデータロード
        self.dfRaceInfo = pd.read_pickle('./data/race_info.pkl')
        self.dfHorseResult = pd.read_pickle('./data/horse_result2.pkl')
        self.dfHorsePed = pd.read_pickle('./data/horse_ped2.pkl')

        # 学習済みモデルの読み込み
        self.model = pickle.load(open('./data/model1.pkl','rb'))

        
    def entryHorse(self, raceinfo, horseinfo):

        #出馬表とレース情報テーブル結合
        dfEntryi = pd.merge(left=horseinfo,right=raceinfo, how='left', on='raceId')
        dfHorseResulti = pd.merge(left=self.dfHorseResult,right=self.dfRaceInfo,
                                how='left', on='raceId',suffixes=['','_i'])
        #戦績テーブルからゴミを除外
        df = dfHorseResulti.copy()
        columns = []
        for cn in df.columns:
            if '_i' not in cn:
                columns.append(cn)
        dfHorseResulti = df[columns]
        #空のデータフレームを生成
        dfEntryH = pd.DataFrame (columns=['horseId','日付'])
        
        indexList = dfEntryi.index
        # #レースを日付降順にソートする
        dfHorseResulti = dfHorseResulti.sort_values(['horseId', '日付'], ascending=[True, False])

        #レース結果のテーブルを1行ずつ取り出す
        for cnt, rowIndex in enumerate(indexList):
            
            try:
                #レース結果から1行分のデータを取り出す
                rowDf = dfEntryi.loc[dfEntryi.index==rowIndex]
                #horseIと日付を取り出す
                horseId = rowDf['horseId'].iloc[0]
                dt = rowDf['日付'].iloc[0]
                
                for idx,tmpDf in \
                    enumerate(dfHorseResulti[(dfHorseResulti['horseId']==horseId)&\
                                            (dfHorseResulti['日付']<dt)].head(9).iterrows()):
                    idxx = idx+1
                    tmpDfx = pd.DataFrame(tmpDf[1]).T
                    #列名にループ回数でサフィックスを付加
                    tmpDfx.columns = tmpDfx.columns + '_' + str(idxx)
                    #結合の為にidexを合わせる
                    tmpDfx.index = rowDf.index
                    #結合の実行
                    rowDf = pd.concat([rowDf, tmpDfx], axis=1)
                #1行分のレコードを結合
                dfEntryH = pd.concat([dfEntryH,rowDf])
            except:
                print('exception catch')
                break

        df = dfEntryH.copy() 
        #スカラー関数の定義
        def CalcInterval(x):
            try:
                interval = datetime.datetime.strptime(x['日付'], '%Y/%m/%d')-\
                            datetime.datetime.strptime(x['日付_1'], '%Y/%m/%d')
                return interval.days
            except:
                return 0
        #apply関数で一括処理
        df['出走間隔'] = df.apply(CalcInterval, axis=1)
        dfEntryp = pd.merge(left=df, right=self.dfHorsePed, how='left', on='horseId')

        #データセットのロード
        #使用する列名を指定
        resultCol = [
            '日付','raceId','枠番','馬番','horseId','性','年齢','斤量',
            'jockeyId','単勝','人気','trainerId','拠点','馬体重','体重増減',
            '出走間隔','ハンデ','着順','R','コース種','コース回り','距離','天気',
            '馬場','開催場所','グレード','制限'
        ]
        recordCol = [
            'R','頭数','枠番','馬番','単勝','人気','着順','jockeyId','斤量',
            'タイム','着差','上り','馬体重','体重増減','出走間隔','コース種',
            'コース回り','距離','天気','馬場','開催場所','グレード','制限','ハンデ'
        ]
        pedCol = ['pedId_' + str(i) for i in range(0,62)]
        #前N走分戦績の列名を生成
        recordCol9 = []
        for i in range(1, 10):
            tmpList = list(map(lambda x: x + '_' + str(i), recordCol))
            recordCol9 += tmpList
        #列名を合体
        COLUMNS = resultCol + recordCol9 + pedCol

        COLUMNS2 = COLUMNS.copy()
        COLUMNS2.remove('着順')
        frame = pd.DataFrame(columns=COLUMNS2)
        targetDataset = pd.concat([frame, dfEntryp])[COLUMNS2]

    #出馬表のラベルエンコーディング
        # #ラベルエンコーディング関数の定義
        def labelEncode(df, target, recflg=False):
            #複数列のラベルエンコーディング関数の定義
            def listEncoder(tdf, le, cols):
                #データフレームのコピー
                tdf_ = tdf.copy()
                #列名から値を取り出す
                encoList = []
                for col in cols:
                    encoList += tdf_[col].unique().tolist()
                #エンコーダーを生成
                le.fit(encoList)
                #複数列分ループ
                for col in cols:
                    #欠損データ以外の列を取り出す
                    notNull = tdf_[col][tdf_[col].notnull()]
                    #エンコード実行してindexをキーにデータフレームに書き込む
                    tdf_[col] = pd.Series(le.transform(notNull), index=notNull.index)
                    #エンコードした列はcategory列に変換
                    tdf_[col] = tdf_[col].astype('category')
                return tdf_, le
            #データフレームのコピー
            tdf = df.copy()
            #ラベルエンコーダーをインスタンス
            le = preprocessing.LabelEncoder()
            #戦績かどうかで分岐
            if not recflg:
                #リストかどうかで分岐
                if type(target) != list:
                    #エンコーダーの生成
                    le.fit(tdf[target])
                    #欠損データ以外の列を取り出す
                    notNull = tdf[target][tdf[target].notnull()]
                    #エンコード実行してindexをキーにデータフレームに書き込む
                    tdf[target] = pd.Series(le.transform(notNull), index=notNull.index)
                    #エンコードした列はcategory列に変換
                    tdf[target] = tdf[target].astype('category')
                else:
                    #戦績以外で複数データだったら複数列エンコードの実行
                    tdf, le = listEncoder(tdf, le, target)
            else:
                #戦績データは列名にサフィックスを付与したリストを生成
                cols9 = [target] + [target + '_' + str(i) for i in range(1, 10)]
                #複数列エンコードの実行
                tdf, le = listEncoder(tdf, le, cols9)
            #データフレームとエンコーダーをreeturn
            return tdf, le

        #データフレームコピー
        df = targetDataset.copy()
        #カテゴリ変数をラベルエンコード
        horseList = ['horseId'] + ['pedId_' + str(i) for i in range(0,62)]

        # print(type(df))
        df, leHorse = labelEncode(df,horseList)
        df, leGender = labelEncode(df,'性')
        df, leTrainer = labelEncode(df,'trainerId')
        df, leHomeBase = labelEncode(df,'拠点')
        df, leJockey = labelEncode(df,'jockeyId',recflg=True)
        df, leHandi = labelEncode(df,'ハンデ',recflg=True)
        df, leType = labelEncode(df,'コース種',recflg=True)
        df, leDir = labelEncode(df,'コース回り',recflg=True)
        df, leWether = labelEncode(df,'天気',recflg=True)
        df, leCondition = labelEncode(df,'馬場',recflg=True)
        df, lePlace = labelEncode(df,'開催場所',recflg=True)
        df, leGrade = labelEncode(df,'グレード',recflg=True)
        df, leRegulation = labelEncode(df,'制限',recflg=True)

        #量的変数の列名を生成
        numericCols = ['年齢']
        cols1 = ['枠番','馬番','単勝','人気','斤量','馬体重',
                '体重増減','出走間隔','R','距離']
        cols2 = ['頭数','着順','タイム','着差','上り']
        numericCols += cols1
        cols3 = cols1 + cols2
        for i in range(1,10):
            numericCols += map(lambda x: x + '_' + str(i),cols3)
        #量的変数に対して片変数を実行
        for col in numericCols:
            df[col] = df[col].astype(float)
        
        #推論
        test = df
        test_x = test.drop(['日付','raceId','単勝'],axis=1)

        return test_x

        #検証データに対して予測実行
        prad = self.model.predict(test_x)
        #結果可視化の為に元のデータセットをコピー
        eval_df = dfEntryi.copy()   #元はdfEntryなってた勘
        #予測結果を列として追加
        eval_df['pred'] = prad
        #1となる確率を列として追加
        eval_df['proba'] = self.model.predict_proba(test_x)[:,1]
        #raceIdでグルーピングで1となる確率が高い順にランク付けを実行
        eval_df['予測着順'] = eval_df.groupby('raceId')['proba'].rank(ascending=False)

        warnings.simplefilter('ignore')

        eval_df['正規化値'] = eval_df.groupby('raceId')['proba']\
            .apply(lambda x: preprocessing.minmax_scale(x)).explode().tolist()
        eval_df[eval_df['raceId']=='202301010301']
        eval_df['標準化分散値'] = eval_df.groupby('raceId')['proba']\
            .apply(lambda x: preprocessing.scale(x)).explode().tolist()
        view_df = eval_df[['raceId','馬名','馬番','枠番','単勝','人気','予測着順',
                        'pred','proba','正規化値','標準化分散値']]

        view_df = view_df.sort_values('予測着順')
        view_df['予測印'] = view_df['pred'].map(lambda x: '◯' if x == 1 else '')

        view_df = view_df[['raceId','馬名','馬番','枠番','予測着順']]
        datas = []
        for dr,dn,db,dw,dy in zip(view_df['raceId'],view_df['馬名'],view_df['馬番'],view_df['枠番'],view_df['予測着順']):
            datas += [dr,dn,int(db),int(dw),int(dy)]

        return datas

In [32]:
pred_instans = Main()

engine = sa.create_engine(
    sa.engine.url.URL.create(
        drivername = "mysql",       # or postgresql
        username = con.USER,        # e.g. "my-database-user"
        password = con.PASSWORD,    # e.g. "my-database-password"
        host = con.HOST,            # e.g. "127.0.0.1"
        port = con.PORT,            # e.g. 3306
        database = con.DB,          # e.g. "my-database-name"
    )
)

race_id = "202301010101"

# race_idのリストを基にスクレイピングを行う
# for race_id in tqdm(raceIdList):
# result_horseテーブルを取得しデータフレームに変換、データの整形
sql_query = f"select * from result_horse where race_id = {race_id};"
result_horse_df = pd.read_sql(sql=sql_query, con=engine)
result_horse_df = result_horse_df.drop('RANKING', axis=1)
result_horse_df["JOCKEY"] = result_horse_df["JOCKEY"].str.replace("▲","").str.replace("△","").str.replace("☆","").str.replace("★","").str.replace("▲","").str.replace("◇","")
result_horse_df.loc[result_horse_df['BASE'] == "美浦", 'BASE'] = "東"
result_horse_df.loc[result_horse_df['BASE'] == "栗東", 'BASE'] = "西"
result_horse_df = result_horse_df.rename(columns={'HNAME':'馬名','HORSEFRAME': '枠番','HORSENUMBER': '馬番','GENDER': '性','RACE_ID': 'raceId','HORSE_ID': 'horseId','AGE':'年齢','WEIGHT':'斤量','JOCKEY_ID':'jockeyId','ODDS':'単勝','POPULAR':'人気','TORAINER_ID':'trainerId','BASE':'拠点','HORSE_WEIGHT':'馬体重','WEIGHT_GAIN_LOSS':'体重増減'})

# raceテーブルを取得しデータフレームに変換、データの整形
sql_query = f"select * from race where race_id = {race_id};"
race_df = pd.read_sql(sql=sql_query, con=engine)
race_df = race_df.drop('HORSE_TOTAL', axis=1)
race_df = race_df.drop('TIME', axis=1)
race_df["RACEDATE"] = race_df["RACEDATE"].str.replace("月","/").str.replace("日","").str.replace("年","/")
race_df = race_df.rename(columns={'RACE_ID': 'raceId','RACENUMBER':'R','RACEDATE': '日付','GROUND':'コース種','SPIN':'コース回り','DISTANCE':'距離','WEATHER':'天気','SITUATION':'馬場','PLACE':'開催場所','GRADE':'グレード','LIMIT':'制限','HANDICAP':'ハンデ'})

# ai予想プログラムを呼び出して返り値を受け取る
pred_list = pred_instans.entryHorse(race_df,result_horse_df)

# ai予想プログラムで帰ってきたリストを引数として渡す
# db_instans.insert("PREDICTION_HORSE",pred_list)



    枠番   馬番 horseId  性   年齢    斤量 jockeyId   人気 trainerId 拠点  ...  pedId_52  \
0  5.0  5.0     304  1  2.0  55.0        4  1.0         2  0  ...       170   
1  8.0  8.0     307  0  2.0  55.0        2  2.0         0  0  ...       254   
2  6.0  6.0     308  1  2.0  55.0        0  6.0         3  1  ...       169   
3  4.0  4.0     306  0  2.0  55.0        7  3.0         5  1  ...       190   
4  1.0  1.0     303  0  2.0  55.0        1  5.0         4  1  ...       177   
5  2.0  2.0     301  1  2.0  55.0        3  7.0         6  1  ...       153   
6  3.0  3.0     302  0  2.0  53.0        5  4.0         7  0  ...       165   
7  7.0  7.0     305  0  2.0  52.0        6  8.0         1  0  ...       277   

   pedId_53  pedId_54 pedId_55  pedId_56 pedId_57 pedId_58  pedId_59 pedId_60  \
0        42       136      241       267       35       10       240       29   
1        79       204      259       113       87      200       258        5   
2       100       168       18        90     